#1 Import statements and global variables assignement and audio generation

In [1]:
import cv2
import numpy as np
import random,os,subprocess
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [22]:
#create an id for video
import os
import json

json_file_path = "data.json"

# Create a JSON file if it doesn't exist
if not os.path.exists(json_file_path):
    with open(json_file_path, 'w') as json_file:
        initial_data = {'last_id': 999}
        json.dump(initial_data, json_file)

# Load data from JSON
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Fetch the last ID, increment it by 1, and attach "Not ready"
last_id = data['last_id']
new_id = last_id + 1

# Update the data
data['last_id'] = new_id
data[str(new_id)] = "Not ready"

# Save the updated data back to JSON
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"New ID: {new_id}")
print("Updated JSON content:")
print(json.dumps(data, indent=4))


New ID: 1000
Updated JSON content:
{
    "last_id": 1000,
    "1000": "Not ready"
}


In [ ]:
# change video status in json file
import os
import json

def update_id_status(json_file_path, target_id, new_status):
    # Load data from JSON
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Update the value of the target ID to the new status
    if str(target_id) in data:
        data[str(target_id)] = new_status
        with open(json_file_path, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"ID {target_id} has been set to '{new_status}'.")
    else:
        print(f"ID {target_id} not found in the JSON.")

In [ ]:
# get status of video
import os
import json

def get_id_status(json_file_path, target_id):
    # Load data from JSON
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Retrieve the status of the target ID
    if str(target_id) in data:
        status = data[str(target_id)]
        return status
    else:
        return f"ID {target_id} not found in the JSON."

In [2]:
ideas_file='file.txt'
lang="en"
with open(ideas_file,'r') as file:
  ideas=file.readlines()
c=0
for i in range(len(ideas)):
  if(i!=len(ideas)-1):
    ideas[i]=ideas[i][0:len(ideas[i])-1]
  c+=len(ideas[i])
print(ideas)
print(c)

['SA Aramco: Largest oil reservoir, 260B barrels, 186T cubic ft gas, top oil co.', 'Saudi Aramco IPO: Domestic listing,']
113


In [3]:
conj_file="details.txt"
with open(conj_file,'r') as file:
  details=file.readlines()
for i in range(len(details)):
  if(i!=len(details)-1):
    details[i]=details[i][0:len(details[i])-1]
print(details)

line_percentage=[0 for i in details]
total_duration=0

['Founded in 1933, Saudi Aramco boasts immense oil and gas reserves, with over 260 billion barrels of oil and 186 trillion cubic feet of natural gas. This puts it ahead of other major oil companies globally and positions it as a significant player in the energy sector.', 'The long-anticipated Saudi Aramco IPO was confirmed in early November.']


##generate audio for test

In [4]:
!pip install gTTS pydub

In [5]:
from gtts import gTTS
import os
from pydub import AudioSegment

output_folder = "/content"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for i in range(len(details)):
  tts = gTTS(text=details[i], lang=lang)
  audio_file_path = os.path.join(output_folder, f"{i}.mp3")
  tts.save(audio_file_path)
  audio = AudioSegment.from_file(f"{i}.mp3", format="mp3")
  duration = len(audio) / 1000
  line_percentage[i]=duration
  total_duration+=duration

In [7]:
import os
from pydub import AudioSegment

audio_folder = "/content"  # Change this to the folder containing your existing audio files

if not os.path.exists(audio_folder):
    print("Output folder does not exist.")
    exit()

total_duration = 0

# Iterate through existing audio files
for i in range(len(details)):
    audio_file_path = os.path.join(audio_folder, f"{i}.mp3")

    if os.path.exists(audio_file_path):
        audio = AudioSegment.from_file(audio_file_path, format="mp3")
        duration = len(audio) / 1000
        total_duration += duration
        line_percentage[i]=duration
    else:
        print(f"Audio file {i}.mp3 does not exist.")

print("Total Duration:", total_duration)

Total Duration: 28.704


##set variables

In [8]:
image_paths = ["image1.jpeg", "image2.jpg", "image3.jpg", "image4.jpeg","image5.jpeg"]
fps=24
#total_duration =  0.5*c+0.5
text_color=(0, 0, 0)
text_bg_color=(255, 200, 0)
text_bg_color=(255, 203, 80)
logo_path = "logo.png"
output_path = f"{new_id}video.mp4"
final_video = f"{new_id}final.mp4"
final_video_with_narration = f"{new_id}final_video.mp4"
print(total_duration,' in seconds')
print(total_duration/60,' in minutes')
print(line_percentage)

28.704  in seconds
0.4784  in minutes
[22.464, 6.24]


#2 Animations

In [9]:
def zoom_in(image_path, output_path, duration=3, fps=30, max_scale=1.15):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Calculate the scale increment per frame
    scale_increment = (max_scale - 1.0) / (duration * fps)

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Create the zoom-out frames
    current_scale = 1.0
    for i in range(int(duration * fps)):
        # Calculate the current scale for this frame
        current_scale += scale_increment

        # Calculate the translation to keep the center fixed
        translation_x = center_x * (1 - current_scale)
        translation_y = center_y * (1 - current_scale)

        # Create a scaling and translation matrix to zoom out the image from the center
        transformation_matrix = np.float32([[current_scale, 0, translation_x], [0, current_scale, translation_y]])

        # Apply the transformation to the image
        zoomed_in_image = cv2.warpAffine(image, transformation_matrix, (width, height))

        # Write the frame to the video
        out.write(zoomed_in_image)

    # Release the video writer
    out.release()


def zoom_out(image_path, output_path, duration=3, fps=30, max_scale=1.15):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Calculate the scale decrement per frame
    scale_decrement = (max_scale - 1.0) / (duration * fps)

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Create the zoom-in frames
    current_scale = max_scale
    for i in range(int(duration * fps)):
        # Calculate the current scale for this frame
        current_scale -= scale_decrement

        # Calculate the translation to keep the center fixed
        translation_x = center_x * (1 - current_scale)
        translation_y = center_y * (1 - current_scale)

        # Create a scaling and translation matrix to zoom in the image from the center
        transformation_matrix = np.float32([[current_scale, 0, translation_x], [0, current_scale, translation_y]])

        # Apply the transformation to the image
        zoomed_out_image = cv2.warpAffine(image, transformation_matrix, (width, height))

        # Write the frame to the video
        out.write(zoomed_out_image)

    # Release the video writer
    out.release()

def translate_left(image_path, output_path, duration=3, fps=30):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculate the width of the visible portion (90% of the image width)
    visible_width = int(width * 0.9)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (visible_width, height))

    # Calculate the total translation distance for the entire duration
    total_translation_distance = width - visible_width

    # Calculate the translation distance per second
    translation_per_second = total_translation_distance / duration

    # Create the translated frames
    for i in range(int(duration * fps)):
        # Calculate the translation distance for this frame
        translation_distance = int(i / fps * translation_per_second)

        # Create a translation matrix to shift the image to the left
        translation_matrix = np.float32([[1, 0, -translation_distance], [0, 1, 0]])

        # Apply the translation to the image
        translated_image = cv2.warpAffine(image, translation_matrix, (visible_width, height))

        # Write the frame to the video
        out.write(translated_image)

    # Release everything if job is finished
    out.release()


def translate_right(image_path, output_path, duration=3, fps=30):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Calculate the width of the visible portion (90% of the image width)
    visible_width = int(width * 0.9)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (visible_width, height))

    # Calculate the total translation distance for the entire duration
    total_translation_distance = width - visible_width

    # Calculate the translation distance per second
    translation_per_second = total_translation_distance / duration

    # Create the translated frames
    frames = []
    for i in range(int(duration * fps)):
        # Calculate the translation distance for this frame
        translation_distance = int(i / fps * translation_per_second)

        # Create a translation matrix to shift the image to the left
        translation_matrix = np.float32([[1, 0, -translation_distance], [0, 1, 0]])

        # Apply the translation to the image
        translated_image = cv2.warpAffine(image, translation_matrix, (visible_width, height))

        # Append the frame to the frames list
        frames.append(translated_image)

    # Append the frames in reverse order to the output video to get the inverse effect
    for frame in reversed(frames):
        out.write(frame)

    # Release everything if job is finished
    out.release()

#3 video creation phase 1:

In [10]:
# Function to concatenate video clips into a single video using moviepy
def concatenate_videos(video_paths, output_path, target_resolution=(1280,720)):
    # Create VideoFileClip objects for each video and resize to the target resolution
    video_clips = [VideoFileClip(video_path).resize(target_resolution) for video_path in video_paths]

    # Concatenate the video clips
    final_clip = concatenate_videoclips(video_clips, method="compose")

    # Write the final video to the output path
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    # Close the video clips
    for video_clip in video_clips:
        video_clip.close()

# Function to create a video with transitions between images
def create_transition_video(image_paths, output_path, total_duration, fps=30, animations=None):
    if not animations:
        animations = [zoom_in, translate_left, zoom_out, translate_right]

    # Create temporary video animations for each image
    video_paths = []
    i=0
    for image_path in image_paths:
      video_path=f'temp_video_for_image_n_{i}.mp4'
      create_image_animation=random.choice(animations)
      create_image_animation(image_path, video_path, total_duration / len(image_paths), fps)
      video_paths.append(video_path)
      i+=1

    # Concatenate the temporary videos to create the final video
    concatenate_videos(video_paths, output_path)

    # Remove temporary video files
    for video_path in video_paths:
        os.remove(video_path)

In [11]:
create_transition_video(image_paths, output_path, total_duration, fps)

Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4



t:  20%|██        | 138/686 [00:14<01:19,  6.87it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file temp_video_for_image_n_0.mp4, 7464960 bytes wanted but 0 bytes read,at frame 137/138, at time 5.71/5.71 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready video.mp4


#4 Video creation phase 2:

In [12]:
import cv2
import numpy as np

def add_light_color_effect(arr, alpha=0.5, beta=0.5):
    # Create a yellow tint image
    yellow_tint = np.zeros_like(arr)
    yellow_tint[:, :, 0] = 0  # Hue: 30 (Yellow)
    yellow_tint[:, :, 1] = 0  # Saturation: 100 (Medium Saturation)
    yellow_tint[:, :, 2] = 3  # Value: 255 (Maximum Brightness)

    # Combine the original frame with the yellow tint using alpha blending
    return cv2.addWeighted(arr, alpha, yellow_tint, beta, 0)

def gaussian_blur(arr, sigma):
    return cv2.GaussianBlur(arr, (0, 0), sigma)

def add_blurred_split_screen_effect(input_video_path, output_video_path, padding_percentage=0.05, blur_sigma=5):
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    left_padding = int(width * padding_percentage)
    right_padding = int(width * padding_percentage)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        left_blurred = frame[:, :left_padding].copy()
        left_blurred = gaussian_blur(left_blurred, sigma=blur_sigma)
        left_blurred = add_light_color_effect(left_blurred)

        right_blurred = frame[:, -right_padding:].copy()
        right_blurred = gaussian_blur(right_blurred, sigma=blur_sigma)
        right_blurred = add_light_color_effect(right_blurred)

        combined_frame = frame.copy()
        combined_frame[:, :left_padding] = left_blurred
        combined_frame[:, -right_padding:] = right_blurred

        out.write(combined_frame)

    cap.release()

    # Release the VideoWriter object after writing all frames
    out.release()
    cv2.destroyAllWindows()

# Example usage:
video_path = output_path
output_path = f"{new_id}output.mp4"
add_blurred_split_screen_effect(video_path, output_path, padding_percentage=0.05, blur_sigma=5)

#Video creation phase 3:

##adding subtitles

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to RGB color space
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to a PIL Image
    pil_image = Image.fromarray(frame)

    # Create a drawing context for the PIL Image
    draw = ImageDraw.Draw(pil_image)

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Add the subtitle to the frame
            font_path = '/content/ReadexPro-Bold.ttf'
            font_size = 48
            font = ImageFont.truetype(font_path, font_size)
            text = ideas[i]
            textsize = font.getsize_multiline(text)

            # Position the subtitle on the left or right side of the frame with a small margin depending on lang variable
            if lang == 'ar':
                textX = frame.shape[1] - 100 - textsize[0]
            else:
                textX = 100
            textY = frame.shape[0] - 150 - textsize[1]

            # Add some padding between the text and its black background
            padding = 10

            # Add a black background for the subtitle
            draw.rectangle((textX - padding, textY + textsize[1] + padding, textX + textsize[0] + padding, textY - padding), fill=(0, 0, 0))

            # Put the subtitle text in color #FFC800
            draw.text((textX, textY), text, fill=(255, 200, 0), font=font)

    # Convert the PIL Image back to a NumPy array
    frame = np.array(pil_image)

    # Convert the frame back to BGR color space
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()


Streaming output truncated to the last 5000 lines.
  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  textsize = font.getsize_multiline(text)

  t

##5 fixing an issue in subtitles

In [18]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# Load the video file
cap = cv2.VideoCapture(output_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the start and end frames for each subtitle
start_frames = [int(sum(line_percentage[:i]) * fps) for i in range(len(line_percentage))]
end_frames = [int(sum(line_percentage[:i+1]) * fps) for i in range(len(line_percentage))]

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(final_video, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# Add the subtitles to the video
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to RGB color space
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to a PIL Image
    pil_image = Image.fromarray(frame)

    # Create a drawing context for the PIL Image
    draw = ImageDraw.Draw(pil_image)

    # Check if we need to add a subtitle to this frame
    for i, (start, end) in enumerate(zip(start_frames, end_frames)):
        if start <= frame_num < end:
            # Add the subtitle to the frame
            font_path = '/content/ReadexPro-Bold.ttf'
            font_size = 48
            font = ImageFont.truetype(font_path, font_size)
            text = ideas[i]
            textsize = font.getsize_multiline(text)

            # Position the subtitle on the left or right side of the frame with a small margin depending on lang variable
            if lang == 'ar':
                textX = frame.shape[1] - 100 - textsize[0]
            else:
                textX = 100
            textY = frame.shape[0] - 150

            # Add some padding between the text and its background
            padding = 10

            # Split the text into wrapped lines if it's too wide for the frame
            max_text_width = frame.shape[1] - 2 * (textX + padding)  # Max available width
            wrapped_lines = []
            words = text.split()
            line = ""
            for word in words:
                test_line = line + word + " "
                test_line_width = font.getsize_multiline(test_line)[0]
                if test_line_width <= max_text_width:
                    line = test_line
                else:
                    wrapped_lines.append(line)
                    line = word + " "
            wrapped_lines.append(line)

            # Calculate the total height of wrapped text
            wrapped_text_height = len(wrapped_lines) * textsize[1]

            # Calculate the starting Y position for wrapped text
            wrapped_text_start_y = textY - wrapped_text_height + textsize[1] + padding

            # Draw each wrapped line of text and background
            for line in wrapped_lines:
                # Calculate the width and height of the text for positioning the black background
                text_width, text_height = font.getsize(line)
                bg_left = textX - padding
                bg_top = wrapped_text_start_y - padding
                bg_right = textX + text_width + padding
                bg_bottom = bg_top + text_height + padding

                # Draw the black background
                draw.rectangle((bg_left, bg_top, bg_right, bg_bottom), fill=text_bg_color)

                # Calculate the position for the text within the black background
                text_position = (textX, wrapped_text_start_y)

                # Draw the text in white
                draw.text(text_position, line, fill=text_color, font=font)

                wrapped_text_start_y += textsize[1] + 2.5*padding  # Move to the next line

    # Convert the PIL Image back to a NumPy array
    frame = np.array(pil_image)

    # Convert the frame back to BGR color space
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Write the frame to the output video
    out.write(frame)
    frame_num += 1

# Release resources
cap.release()
out.release()


Streaming output truncated to the last 5000 lines.
  textsize = font.getsize_multiline(text)

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  test_line_width = font.getsize_multiline(test_line)[0]

  text_width, text_height = font.getsize(line)

  textsize = font.getsize_multiline(text)

  t

##6 Adding audio

In [14]:
from moviepy.editor import AudioFileClip, VideoFileClip, concatenate_audioclips

# Load the video file with subtitles
video_clip = VideoFileClip(final_video)

audio_clips = []
for i in range(len(line_percentage)):
    audio_clip = AudioFileClip(f"{i}.mp3")
    audio_clips.append(audio_clip)

concatenated_audio = concatenate_audioclips(audio_clips)

video_clip = video_clip.set_audio(concatenated_audio)

# Write the final video with the new audio
video_clip.write_videofile(final_video_with_narration, codec="libx264", audio_codec="aac")

Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_video.mp4



Moviepy - Done !
Moviepy - video ready final_video.mp4


In [15]:
import os

for i in range(len(line_percentage)):
  os.remove(f'{i}.mp3')

#7 the next cell adds the Logo

In [16]:
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip

def add_logo_to_video(video_path, logo_path, output_path, logo_scale=0.1, padding_left=20):
    # Load the video clip
    video_clip = VideoFileClip(video_path)

    # Load the logo image and resize it
    logo = ImageClip(logo_path)
    logo = logo.resize(height=int(video_clip.h * logo_scale))

    # Set the duration of the logo image to match the duration of the video
    logo = logo.set_duration(video_clip.duration)

    # Calculate the position of the logo to create left padding
    logo_x_pos = padding_left
    logo_y_pos = "top"

    # Set the position of the logo with padding on the left
    logo = logo.set_position((logo_x_pos, logo_y_pos))

    # Composite the logo on top of the video
    final_clip = CompositeVideoClip([video_clip, logo])

    # Write the final video with the logo to a file
    final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

    # Close the video clip and free up resources
    final_clip.close()
    video_clip.close()



output_path = f"{new_id}.mp4"  # Replace with the desired output path

add_logo_to_video(final_video_with_narration, logo_path, output_path)


Moviepy - Building video fi_vid.mp4.
MoviePy - Writing audio in fi_vidTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video fi_vid.mp4



t: 100%|█████████▉| 687/690 [00:46<00:00, 12.84it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file final_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame 686/690, at time 28.58/28.72 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|█████████▉| 689/690 [00:46<00:00, 12.89it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file final_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame 688/690, at time 28.67/28.72 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready fi_vid.mp4


#**Prompt for generating main points from article:**
Please extract the main points from the provided article while following these guidelines :

Ensure that the main points create a coherent narrative when someone reads them in order.
Each main point in 84 characters (including spaces) or fewer, and enclose it in squared brackets [].
Make at least 10 main points.

mainpoints should be outputed this this :
[ main point 1 here ]
[ main point 2 here ]
...
[main  point n here ]

after writing all the main points, for each main points make a well detailed explanation enclosed within curly brackets {} for the main point to capture all nuances of the original article.



article text :